In [1]:
import PyLidar3
import os
import re
import csv
import math
import numpy as np
from pynq import allocate
from pynq import Overlay
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import time

ol = Overlay('/home/xilinx/pynq/overlays/clusterOp2/clusterOp2.bit')

angle = np.arange(0, 360)

port = "/dev/ttyUSB0" #linux
Obj = PyLidar3.YdLidarX4(port, 12000) 

# To use with csv file instead of LIDAR sensor, make sure 
# there is a folder called data in the same working directory
# and there is a CSV file called "data.csv" in that folder

#with open('./data/data.csv') as lidarData:
#    lidar_reader = csv.reader(lidarData)
#        
#    for point in lidar_reader:
#        distance.append(np.uint32(point[1]))

In [2]:
if(Obj.Connect()):
    print(Obj.GetDeviceInfo())
else:
    print("Error connecting to device")

{'model_number': '6', 'firmware_version': '1.5', 'hardware_version': '1', 'serial_number': '201904120'}


In [3]:
gen = Obj.StartScanning()
data = next(gen)
distance = []
for points in data:
    distance.append(data[points])
#print(distance)
Obj.StopScanning()

In [4]:
Obj.Disconnect()

In [5]:
#Processes the raw data into objects, removes noise
def clusterOp(angle, distance, D_th = 100, minGroupSize = 4):
    objects = []
    grouped = set()

    for point in range(0,360):

        #If point has already been grouped move onto the next point
        if point in grouped:
            continue

        cluster = set()
        cluster.add(point)

        #Test in counter clockwise direction
        for i in range(0,360):
            dTheta = angle[i] - angle[point]
            d1 = distance[point]
            d2 = distance[i]
            d3 = math.sqrt(d1**2 + d2**2 - 2*d1*d2*math.cos(dTheta))

            
            if d3 < D_th:
                cluster.add(i)
                grouped.add(i)
                point = i

        if len(cluster) >= minGroupSize:
            objects.append(cluster)

    return objects

In [6]:
ol.clusterOp2_0.register_map
dma = ol.axi_dma
input_buffer = allocate(shape=(360,), dtype = np.int32)
np.copyto(input_buffer, distance)
output_buffer = allocate(shape=(380,), dtype=np.int32)
#print(input_buffer)

In [7]:
start = time.time()

CONTROL_REGISTER = 0x0
ol.clusterOp2_0.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma.sendchannel.transfer(input_buffer)
dma.sendchannel.wait()
dma.recvchannel.transfer(output_buffer)

clusterOp_data = output_buffer

del input_buffer, output_buffer

cluster_n = []  # initialize empty list for first cluster
clusters = []  # initialize list to hold all clusters

for value in clusterOp_data:
    if value == 720:
        if cluster_n:  # check if cluster_n already has values
            clusters.append(cluster_n)  # if so, add it to clusters
            cluster_n = []  # reset cluster_n to empty list
    else:
        cluster_n.append(value)  # add value to current cluster

if cluster_n:  # handle the last cluster after the loop ends
    clusters.append(cluster_n)
    
end = time.time()

print(end-start)

print(clusters)  # display all clusters
    

0.010308980941772461
[[1, 2, 3, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 341, 342, 343, 344, 345, 346, 347, 348, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359], [25, 27, 28, 29, 30, 31, 32, 33, 34, 37, 40, 53], [38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], [55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72], [81, 88, 91, 92, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 120, 122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 171, 174], [170, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 

In [8]:
start = time.time()
clusters = clusterOp(angle, distance)
end = time.time()
print(end-start)
print(clusters)  # display all clusters


1.7361619472503662
[{224, 4, 230, 199, 136, 236, 174, 180, 155}, {165, 8, 265, 202, 171, 177, 115, 183, 159}, {129, 35, 167, 104, 10, 205, 79, 148, 186}, {96, 128, 98, 122, 78, 241, 116, 253, 90, 93, 95}, {257, 194, 169, 238, 81, 188, 125}, {170, 107, 208, 151, 88, 189}, {224, 230, 199, 136, 236, 174, 180, 155, 92}, {256, 226, 100, 264, 269, 270, 242, 243, 244, 254, 246, 247, 248, 94, 255}, {128, 97, 103, 109, 241, 122, 253}, {224, 99, 230, 199, 105, 236, 111, 180, 117}, {120, 101, 126, 239}, {209, 146, 190, 102}, {257, 194, 169, 106, 238, 188, 125}, {265, 202, 108, 177, 114, 183}, {128, 97, 103, 109, 110, 241, 122, 253}, {257, 194, 238, 175, 112, 156}, {257, 194, 238, 113, 182, 188}, {96, 128, 98, 122, 78, 241, 116, 118, 253, 90, 93, 95}, {96, 128, 98, 122, 78, 241, 116, 119, 253, 90, 93, 95}, {96, 128, 98, 122, 78, 241, 116, 121, 90, 93, 253, 95}, {128, 98, 241, 116, 122, 123, 253}, {224, 99, 230, 199, 105, 236, 111, 180, 117, 124}, {96, 128, 98, 122, 78, 241, 116, 253, 90, 127, 93, 